References:

1. https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py
2. https://pywavelets.readthedocs.io/en/latest/

In [1]:
import os
import sys
import torch
import argparse
import torchvision
import numpy as np
import torch.nn as nn
import scipy.io as sio
import cv2
import pywt
import pywt.data

from PIL import Image
from tqdm import tqdm
from datetime import datetime
from collections import defaultdict
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from os.path import join, isdir, abspath, dirname
import matplotlib.pyplot as plt
# Customized import.
from networks import HED
from datasets import BsdsDataset
from utils import Logger, AverageMeter, \
    load_checkpoint, save_checkpoint, load_vgg16_caffe, load_pretrained_caffe


# # Parse arguments.
# parser = argparse.ArgumentParser(description='HED training.')
# # 1. Actions.
# parser.add_argument('--test',             default=False,             help='Only test the model.', action='store_true')
# # 2. Counts.
# parser.add_argument('--train_batch_size', default=1,    type=int,   metavar='N', help='Training batch size.')
# parser.add_argument('--test_batch_size',  default=1,    type=int,   metavar='N', help='Test batch size.')
# parser.add_argument('--train_iter_size',  default=10,   type=int,   metavar='N', help='Training iteration size.')
# parser.add_argument('--max_epoch',        default=40,   type=int,   metavar='N', help='Total epochs.')
# parser.add_argument('--print_freq',       default=500,  type=int,   metavar='N', help='Print frequency.')
# # 3. Optimizer settings.
# parser.add_argument('--lr',               default=1e-6, type=float, metavar='F', help='Initial learning rate.')
# parser.add_argument('--lr_stepsize',      default=1e4,  type=int,   metavar='N', help='Learning rate step size.')
# # Note: Step size is based on number of iterations, not number of batches.
# #   https://github.com/s9xie/hed/blob/94fb22f10cbfec8d84fbc0642b224022014b6bd6/src/caffe/solver.cpp#L498
# parser.add_argument('--lr_gamma',         default=0.1,  type=float, metavar='F', help='Learning rate decay (gamma).')
# parser.add_argument('--momentum',         default=0.9,  type=float, metavar='F', help='Momentum.')
# parser.add_argument('--weight_decay',     default=2e-4, type=float, metavar='F', help='Weight decay.')
# # 4. Files and folders.
# parser.add_argument('--vgg16_caffe',      default='',                help='Resume VGG-16 Caffe parameters.')
# parser.add_argument('--checkpoint',       default='',                help='Resume the checkpoint.')
# parser.add_argument('--caffe_model',      default='',                help='Resume HED Caffe model.')
# parser.add_argument('--output',           default='./output',        help='Output folder.')
# parser.add_argument('--dataset',          default='./data/HED-BSDS', help='HED-BSDS dataset folder.')
# # 5. Others.
# parser.add_argument('--cpu',              default=False,             help='Enable CPU mode.', action='store_true')
# args = parser.parse_args()

# Set device.
device = torch.device('cuda')



In [2]:
def wav_trans_grayscale(images):
    images = torchvision.utils.make_grid(images)
    npimg = images.numpy()
    img_np_right_dim = np.transpose(npimg, (1, 2, 0))
    img_np_right_dim = cv2.cvtColor(img_np_right_dim, cv2.COLOR_BGR2GRAY)
    coeffs2 = pywt.dwt2(img_np_right_dim, 'haar')
    LL, (LH, HL, HH) = coeffs2
    return([LL, LH, HL, HH])

def im_save_path_modifier(input_path, mode):
    """
    parameter: input_path is the path of the original image
    parameter: mode is one of LL, LH, HL, HH
    """
    dir_path = input_path[:16] + f'{mode}/' + input_path[16:input_path.rfind('/')]
    image_path = input_path[:16] + f'{mode}/' + input_path[16:]
    return(dir_path, image_path)

def imshow(img):
#     img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [3]:
# current_dir = abspath(dirname(__file__))
# output_dir = join(current_dir, args.output)
# if not isdir(output_dir):
#     os.makedirs(output_dir)

# # Set logger.
# now_str = datetime.now().strftime('%y%m%d-%H%M%S')
# log = Logger(join(output_dir, 'log-{}.txt'.format(now_str)))
# sys.stdout = log  # Overwrite the standard output.

################################################
# II. Datasets.
################################################
# Datasets and dataloaders.
train_dataset = BsdsDataset(dataset_dir='./data/HED-BSDS', split='train')
test_dataset  = BsdsDataset(dataset_dir='./data/HED-BSDS', split='test')
train_loader  = DataLoader(train_dataset, batch_size=1,
                           num_workers=4, drop_last=True, shuffle=True)
test_loader   = DataLoader(test_dataset,  batch_size=1,
                           num_workers=4, drop_last=False, shuffle=False)


In [4]:
train_dataset.__len__()

28800

In [5]:
dataiter = iter(train_loader)
images, labels, path, label_path = dataiter.next()

wav_trans_imlist = wav_trans_grayscale(images)
wav_trans_imlist_names  = ['LL', 'LH', 'HL', 'HH']

for i in range(len(wav_trans_imlist)):
    
    # saving image
    
    dir_path, image_path = im_save_path_modifier(path[0], str(wav_trans_imlist_names[i]))
    os.makedirs(dir_path)
    plt.imsave(image_path, wav_trans_imlist[i], cmap = 'gray')
    
    # saving label
    
    label_dir_path, label_path = im_save_path_modifier(label_path[0], str(wav_trans_imlist_names[i]))
    os.makedirs(label_dir_path)
    plt.imsave(label_path, labels, cmap = 'gray')


ValueError: object too deep for desired array

We should convert to numpy before saving images and labels.

In [15]:
images.numpy()

array([[[[-104.00699  ,    0.9930121,   12.993012 , ...,   46.99301  ,
            35.99301  ,   23.993011 ],
         [-104.00699  ,   -2.006988 ,   10.993012 , ...,   45.99301  ,
            33.99301  ,   51.99301  ],
         [-104.00699  ,   -7.006988 ,    8.993012 , ...,   45.99301  ,
            25.993011 ,   52.99301  ],
         ...,
         [-104.00699  ,  -87.00699  ,  -90.00699  , ...,  -56.00699  ,
           -73.00699  ,  -66.00699  ],
         [-104.00699  ,  -87.00699  ,  -90.00699  , ...,  -72.00699  ,
           -69.00699  ,  -59.00699  ],
         [-104.00699  ,  -84.00699  ,  -87.00699  , ...,  -60.00699  ,
           -70.00699  ,  -74.00699  ]],

        [[-106.66877  ,   14.331232 ,   30.331232 , ...,   75.33123  ,
            64.33123  ,   51.331234 ],
         [-111.66877  ,   11.331232 ,   28.331232 , ...,   74.33123  ,
            62.331234 ,   79.33123  ],
         [-116.66877  ,    6.3312325,   26.331232 , ...,   74.33123  ,
            54.331234 ,   80.3312

In [ ]:
imshow(torchvision.utils.make_grid(images))

Preprocessing:  
Option 1:

1. Take wavelet transform of every channel
2. Stack up RGB channels after performing wavelet transform
3. Feed this to network.

Option 2:

1. Take grayscale of the input image
2. Pass this to network.  
Note: For option 2, we will have to change the first layer of the network to suit our changed dimensions of input.

In [ ]:
# fig = plt.figure(figsize = [10,10])
# ax1 = fig.add_subplot(221)
# ax1.imshow(LL, cmap = 'gray')
# ax1.set_title('LL')
# ax2 = fig.add_subplot(222)
# ax2.imshow(LH, cmap = 'gray')
# ax2.set_title('LH')
# ax3 = fig.add_subplot(223)
# ax3.imshow(HL, cmap = 'gray')
# ax3.set_title('HL')
# ax4 = fig.add_subplot(224)
# ax4.imshow(HH, cmap = 'gray')
# ax4.set_title('HH')

1. Write a function that does this - Done!
2. Create 4 datasets
3. Train 4 networks

Step 2:

We need to:

1. Load image from path normal/p1  
2. Preprocess image
3. Save it in LL/p1, LH/p1, HL/p1, and HH/p1

1. Use os.mkdir for one image - Done!
2. Do it for every mode. - Done!
3. Do it for every image. - Pending!

#### Tasks for 11/28
1. solve np shape error
2. do it for every image